In [1]:
import pandas as pd
from datasets import load_dataset, Audio


df = pd.read_csv("./data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")
commonvoice = load_dataset("audiofolder", data_dir=f"./data/cv-corpus-21.0-delta-2025-03-14/validated_clips")
commonvoice = commonvoice.cast_column("audio", Audio(sampling_rate=16000)) 



/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import AutoProcessor, Wav2Vec2Model
import torch
import librosa
import soundfile
import pickle
from tqdm import tqdm
import numpy as np
model_checkpoint = "facebook/wav2vec2-base-960h"
model = Wav2Vec2Model.from_pretrained(model_checkpoint)
processor = AutoProcessor.from_pretrained(model_checkpoint)

clip_path = "./data/cv-corpus-21.0-delta-2025-03-14/en"
output_dict = {}
for cv in tqdm(commonvoice["train"]):
    # audio file is decoded on the fly
    inputs = processor(cv["audio"]["array"], sampling_rate=cv["audio"]["sampling_rate"], return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state # i think these are the embeddings that I am going to want
    output_dict[cv['audio']['path'].split("/")[-1].split(".")[0]] = np.mean(last_hidden_states[0].numpy(), axis=0)

with open("commonvoice_embeddings.pkl", 'wb') as file:
    # Use pickle.dump() to serialize and save the dictionary
    pickle.dump(output_dict, file)
    file.close()


/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'facebook/hubert-base-ls960'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/hubert-base-ls960' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.